In [25]:
import crawl_data
import os
import setting
import pandas as pd
from bokeh.plotting import figure 
from bokeh.io import show
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.transform import factor_cmap
from bokeh.palettes import Viridis256
from bokeh.models.annotations import Span




""" 数据读取及预处理 """
# 读取最新数据
path = setting.DATA_PATH
file = os.listdir(path)[-1]
data = pd.read_csv(path + str(file)) 
# 截取数据

# 选取需要展示的数据字段
old_columns = [
    '转债名称',
    '转债价格',
    '转股溢价率',
    "税前收益率", 
    "回售年限",
    "税前回售收益",
    '信用'
]

new_columns = [
    'bond_name',
    'bond_price',
    'to_stock_premium_rate',
    'income_before_taxes_rate',
    'back_to_sell_years',
    'income_tosell_before_taxes',
    'credit',  
]

def to_days(x):
    """ 回售年限 """
    x = str(x)
    if "回售中" in x:
        return  float(0)
    elif "无权" in x:
        return float(6)
    else:
        if "天" in x:
            med = x[0:-1]
            if "年" in med:
                med_list = med.split("年")
                year = float(med_list[0])
                day = float(med_list[1])
                return year*365 + day
            else:
                return float(med)
        else:
            try:
                return float(x[:-1])*365
            except:
                return float(0)

def credit(x):
    """ 信用等级 """
    num = len(x)
    if  '+' in x:
        res = (num -1) * 100 + 75
    elif '-' in x:
        res = (num -1) * 100 + 25
    else:
        res = num * 100 + 50
    return res

def percent_(x):
    """ 百分比 """
    x  = str(x)
    if "回售中" in x or "无" in x:
        return 0
    else:
        try:
            return float(x[0:-1])
        except:
            return 0


# 截取数据
df =  data[old_columns]

# 数据转化
df.loc[:,'转债名称'] = df['转债名称'].apply(lambda x:x[:4])
df.loc[:,'转股溢价率'] = df['转股溢价率'].apply(percent_)
df.loc[:,"税前收益率"] = df["税前收益率"].apply(percent_)
df.loc[:,"回售年限"] = df["回售年限"].apply(to_days)
df.loc[:,"税前回售收益"] = df["税前回售收益"].apply(percent_)
df.loc[:,'信用'] = df['信用'].apply(credit)

# df.columns = new_columns

# df.loc[:,'income_before_taxes_rate'] = df['income_before_taxes_rate'].apply(lambda x: '%.2f%%' % (x))
# df.loc[:,'to_stock_premium_rate'] = df['to_stock_premium_rate'].apply(lambda x: '%.2f%%' % (x))

""" 数据展示 """
# 传入DataFrame数据
# source = ColumnDataSource(df)

' 数据展示 '

In [17]:
df.head()

,bond_name,bond_price,to_stock_premium_rate,income_before_taxes_rate,back_to_sell_years,income_tosell_before_taxes,credit
0,福特转债,124.12,13.7,-1.26,1434.45,-4.31,250
1,顺丰转债,115.20,26.5,-0.91,1434.45,-2.90,350
2,常汽转债,109.55,6.7,1.84,1434.45,-1.11,225
3,白电转债,103.54,10.2,1.83,1430.80,0.01,250
4,利德转债,113.71,20.9,0.28,1430.80,-2.16,275


In [18]:
df.describe()

,bond_price,to_stock_premium_rate,income_before_taxes_rate,back_to_sell_years,income_tosell_before_taxes,credit
count,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000
mean,113.594742,25.070619,0.323660,943.436005,-3.022320,252.706186
std,15.805320,27.190859,2.902349,412.064842,6.045099,42.688427
min,89.996000,-11.800000,-13.100000,0.000000,-45.200000,175.000000
25%,103.730000,8.600000,-1.210000,734.250000,-4.835000,225.000000
50%,110.458000,18.400000,0.580000,1071.275000,-1.725000,250.000000
75%,119.692500,30.775000,2.290000,1217.275000,0.000000,250.000000
max,209.001000,166.000000,5.700000,1704.550000,11.000000,350.000000


In [ ]:
# 图形配置文件
options = dict(plot_width=1200, plot_height=800,
               x_axis_label="回售年限",y_axis_label= "税前回售收益率",
               y_axis_type = "log" ,
               tools="pan,wheel_zoom,box_zoom,box_select,reset")
TOOLTIPS = [
    ("转债名称", "@bond_name"),
    ('转股溢价率(%)','@to_stock_premium_rate'),
    ("税前收益率(%)", '@income_before_taxes_rate'),
    ("回售天数(天)",'@back_to_sell_years'),
    ("税前回售收益(%)",'@income_tosell_before_taxes'), 
    ('信用','@credit'),
]

# 信用色彩
color_mapper = LinearColorMapper(palette=Viridis256, low=df["credit"].min(), high=df["credit"].max())
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0), title='Weight')

p3 = figure(title="回售年限-税前回售收益率",tooltips=TOOLTIPS,**options)
p3.circle('back_to_sell_years', "income_tosell_before_taxes", color={'field': 'credit', 'transform': color_mapper}, size=12, alpha=0.6,source=source,line_width=4,fill_color="white")
# p3.add_layout(color_bar, 'right')

show(p3)

In [22]:
from bokeh.models.annotations import Label
from bokeh.plotting import figure

p = figure(x_range=(0,10), y_range=(0,10))
p.circle([2, 5, 8], [4, 7, 6], color="olive", size=10)

label = Label(x=5, y=7, x_offset=12, text="Second Point", text_baseline="middle")
p.add_layout(label)

show(p)

In [23]:
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet


source = ColumnDataSource(data=dict(
    temp=[166, 171, 172, 168, 174, 162],
    pressure=[165, 189, 220, 141, 260, 174],
    names=['A', 'B', 'C', 'D', 'E', 'F']))

p = figure(x_range=(160, 175))
p.scatter(x='temp', y='pressure', size=8, source=source)
p.xaxis.axis_label = 'Temperature (C)'
p.yaxis.axis_label = 'Pressure (lbs)'

labels = LabelSet(x='temp', y='pressure', text='names', level='glyph',
                  x_offset=5, y_offset=5, source=source, render_mode='canvas')


p.add_layout(labels)

show(p)

In [26]:
df = df[df["税前收益率"]>0]

In [27]:
df["税前收益率"]

2      1.84
3      1.83
4      0.28
6      2.52
7      1.78
       ... 
185    5.63
186    3.57
187    2.90
188    2.98
190    1.58
Name: 税前收益率, Length: 117, dtype: float64

In [28]:
import numpy as np
from bokeh.models.annotations import BoxAnnotation

x = np.linspace(0, 20, 200)
y = np.sin(x)

p = figure(y_range=(-2, 2))
p.line(x, y)

# region that always fills the top of the plot
upper = BoxAnnotation(bottom=1, fill_alpha=0.1, fill_color='olive')
p.add_layout(upper)

# region that always fills the bottom of the plot
lower = BoxAnnotation(top=-1, fill_alpha=0.1, fill_color='firebrick')
p.add_layout(lower)

# a finite region
center = BoxAnnotation(top=0.6, bottom=-0.3, left=7, right=12, fill_alpha=0.1, fill_color='navy')
p.add_layout(center)

show(p)

In [30]:
file = os.listdir(path)[-1]
file

'2019-12-14 12:14:51.712353.csv'

In [31]:
os.listdir(path)

['2019-11-30 21:03:29.662738.csv',
 '2019-12-04 11:48:11.700961.csv',
 '2019-11-30 15:23:21.555617.csv',
 '2019-11-21 20:44:51.712370.csv',
 '2019-11-26 21:33:12.054369.csv',
 '2019-12-02 22:56:01.909806.csv',
 '2019-11-27 14:18:54.514295.csv',
 '2019-12-03 00:16:58.525212.csv',
 '2019-12-05 09:11:20.015181.csv',
 '2019-11-28 13:56:37.143664.csv',
 '2019-12-05 09:11:24.934799.csv',
 '2019-12-14 12:12:50.629908.csv',
 '2019-12-04 11:48:06.632870.csv',
 '2019-11-30 21:03:24.573295.csv',
 '2019-11-30 20:51:17.431367.csv',
 '2019-12-03 00:17:13.754328.csv',
 '2019-12-13 15:00:56.491699.csv',
 '2019-12-14 12:14:51.712353.csv']